In [22]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta

sample_path = 'paper_data/XBTUSD_'
date = dt(2020,1,1)

query = {
        'ccy': ['XBTUSD'],
        'start_date': 20200101,
        'end_date': 20230102
    }

#cancels at 01-14 as the file does not exist
def merge_files(sample_path: str, start_date: dt, end_date: dt):
    date = start_date
    while date <= end_date:
        path = sample_path + date.strftime("%Y-%m-%d") + '.csv.xz'
        print(path)
        if date == start_date:
            lob_df = pd.read_csv(path)
            if 'system_time' not in lob_df.columns and 'Unnamed: 0' in lob_df.columns:
                lob_df = lob_df.rename(columns={'Unnamed: 0': 'system_time'})
            lob_df.system_time = lob_df.system_time.str.slice(0,19)
        else:
            try:
                tmp = pd.read_csv(path)
            except Exception as e:
                print(e)
                date += timedelta(1)
                continue
                #return lob_df
            tmp = tmp.rename(columns={'Unnamed: 0':'system_time'})
            tmp.system_time = tmp.system_time.str.slice(0,19) #cuts milisecs from date string
            lob_df = pd.concat([lob_df, tmp], ignore_index=True)
        date += timedelta(1)
    return lob_df


lob_df = merge_files(sample_path, dt(2020,1,9), dt(2020,1,20))
#lob_df.to_csv('XBTESD_2020-01-01_2020-01-08_merge.csv.xz', compression='xz')


paper_data/XBTUSD_2020-01-09.csv.xz
paper_data/XBTUSD_2020-01-10.csv.xz
paper_data/XBTUSD_2020-01-11.csv.xz
paper_data/XBTUSD_2020-01-12.csv.xz
paper_data/XBTUSD_2020-01-13.csv.xz
paper_data/XBTUSD_2020-01-14.csv.xz
[Errno 2] No such file or directory: 'paper_data/XBTUSD_2020-01-14.csv.xz'
paper_data/XBTUSD_2020-01-15.csv.xz
paper_data/XBTUSD_2020-01-16.csv.xz
paper_data/XBTUSD_2020-01-17.csv.xz
paper_data/XBTUSD_2020-01-18.csv.xz
paper_data/XBTUSD_2020-01-19.csv.xz
paper_data/XBTUSD_2020-01-20.csv.xz


In [23]:
lob_df.to_csv('XBTUSD_20200109_20200120_merge.csv.xz', compression='xz', index=False)

In [64]:
import pandas as pd

def drop_rows_by_percentage_change(df, column_name, threshold):
    # Create a copy of the dataframe to avoid modifying the original dataframe
    df_copy = df.copy()

    # Calculate the percentage change for the specified column
    percentage_change = df_copy[column_name].pct_change()

    # Drop the rows where the percentage change is less than the threshold
    df_copy = df_copy[percentage_change.abs() >= threshold]

    # Reset the index of the modified dataframe
    df_copy = df_copy.reset_index(drop=True)

    return df_copy

In [65]:
filename = 'paper_data/XBTUSD_2020-01-01.csv.xz'
testfile = pd.read_csv(filename, compression='xz' if 'xz' in filename else None)
testfile_price = drop_rows_by_percentage_change(testfile, 'midpoint', 0.0001)
filenames = filename.split('.')
filenames[0]+= '_price'
filename = ".".join(filenames)
testfile_price.to_csv(filename, compression='xz' if 'xz' in filename else None, index=False)

In [89]:
from datetime import datetime as dt
from datetime import timedelta
input_paths = []
output_paths = []
date = dt(2020,1,1)
while date <= dt(2020,2,29):
    input_paths.append('paper_data/XBTUSD_'+date.strftime("%Y-%m-%d") + '.csv.xz')
    output_paths.append('paper_data_price/XBTUSD_'+date.strftime("%Y-%m-%d") + '_price.csv.xz')
    date += timedelta(1)
# del Jan 14 and Feb 9 as the file doesnt exist
del input_paths[13]
del output_paths[13]
del input_paths[38]
del output_paths[38]


In [91]:
for input_path, output_path in zip(input_paths, output_paths):
    tmp_df = pd.read_csv(input_path, compression='xz')
    tmp_df_price = drop_rows_by_percentage_change(tmp_df, 'midpoint', 0.0001)
    tmp_df_price.to_csv(output_path, compression='xz')

### Columns
'system_time', 
'midpoint', 'spread', 'buys', 'sells', 'bids_distance_x', 'bids_notional_x', 'bids_cancel_notional_x', 'bids_limit_notional_x', 'bids_market_notional_x', 'asks_distance_x', 'asks_notional_x', 'asks_cancel_notional_x', 'asks_limit_notional_x', 'asks_market_notional_x'

In [5]:
file[['system_time','midpoint','spread','buys','sells']]

,system_time,midpoint,spread,buys,sells
0,2019-12-31 20:00:00.279922+00:00,7141.75,0.5,0.0,0.0
1,2019-12-31 20:00:01.279922+00:00,7141.75,0.5,0.0,0.0
2,2019-12-31 20:00:02.279922+00:00,7141.75,0.5,0.0,0.0
3,2019-12-31 20:00:03.279922+00:00,7141.75,0.5,4845.0,118233.0
4,2019-12-31 20:00:04.279922+00:00,7141.75,0.5,0.0,342744.0
...,...,...,...,...,...
86392,2020-01-01 19:59:55.121494+00:00,7220.25,0.5,0.0,0.0
86393,2020-01-01 19:59:56.121494+00:00,7220.25,0.5,0.0,0.0
86394,2020-01-01 19:59:57.121494+00:00,7220.25,0.5,0.0,0.0
86395,2020-01-01 19:59:58.121494+00:00,7220.25,0.5,0.0,0.0


In [5]:
file[file.spread > 0]

,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,...,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
0,2023-04-10 10:10:51.090444+00:00,28348.515,0.01,0.000000,0.000000,-1.763761e-07,-5.291283e-07,-0.000006,-0.000007,-0.000007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-04-10 10:10:52.090444+00:00,28348.515,0.01,0.000000,0.000000,-1.763761e-07,-5.291283e-07,-0.000006,-0.000007,-0.000007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-04-10 10:10:53.090444+00:00,28348.515,0.01,0.000000,0.000000,-1.763761e-07,-5.291283e-07,-0.000006,-0.000007,-0.000007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-04-10 10:10:54.090444+00:00,28348.515,0.01,0.000000,0.000000,-1.763761e-07,-5.291283e-07,-0.000006,-0.000007,-0.000007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-04-10 10:10:55.090444+00:00,28348.515,0.01,0.000000,0.000000,-1.763761e-07,-5.291283e-07,-0.000006,-0.000007,-0.000007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,2023-04-10 10:28:24.090444+00:00,28322.855,0.31,0.000000,41945.657989,-5.472612e-06,-5.825684e-06,-0.000067,-0.000073,-0.000076,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1054,2023-04-10 10:28:25.090444+00:00,28322.705,0.01,0.000000,0.000000,-1.765368e-07,-5.296104e-07,-0.000010,-0.000057,-0.000058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1055,2023-04-10 10:28:26.090444+00:00,28322.705,0.01,0.000000,0.000000,-1.765368e-07,-5.296104e-07,-0.000010,-0.000055,-0.000056,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1056,2023-04-10 10:28:27.090444+00:00,28322.705,0.01,0.000000,0.000000,-1.765368e-07,-5.296104e-07,-0.000010,-0.000057,-0.000057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
demo = pd.read_csv('demo_LTC-USD_20190926.csv.xz', compression='xz')

In [3]:
tmp = {"test1": 1, "test2": 3}
tmp2 = {"test3": 5, "test4": 6}
{**tmp, **tmp2}

{'test1': 1, 'test2': 3, 'test3': 5, 'test4': 6}

In [9]:
from datetime import datetime as dt
str(dt.now()).split()[0]

'2023-05-22'

In [5]:
orderbook_snapshot = {'symbol': 'XBTUSD', 'bids': [[27151.0, 38000.0], [27150.0, 18300.0], 
[27149.5, 11100.0], [27149.0, 17800.0], [27148.5, 15100.0], [27148.0, 23800.0], 
[27147.5, 7300.0], [27147.0, 5800.0], [27146.5, 6400.0], [27146.0, 6700.0], 
[27145.5, 10300.0], [27143.5, 31200.0], [27143.0, 51200.0], [27142.5, 22400.0], 
[27142.0, 18600.0], [27141.5, 30100.0], [27140.5, 41000.0], [27140.0, 107100.0], 
[27139.5, 78700.0], [27139.0, 68800.0]], 'asks': [[27151.5, 30300.0], 
[27154.0, 100.0], [27155.0, 1900.0], [27155.5, 3000.0], [27156.5, 17800.0], 
[27157.0, 24700.0], [27157.5, 50900.0], [27158.0, 17900.0], [27158.5, 10500.0], 
[27159.0, 11000.0], [27159.5, 17800.0], [27160.0, 18000.0], [27160.5, 10000.0], 
[27161.0, 17400.0], [27161.5, 28700.0], [27162.5, 17800.0], [27163.0, 59300.0], 
[27163.5, 103500.0], [27164.0, 59800.0], [27164.5, 22600.0]], 'timestamp': None, 
'datetime': None, 'nonce': None}

In [10]:
best_bid, best_ask = orderbook_snapshot['bids'][0][0], orderbook_snapshot['asks'][0][0]
midpoint = (best_bid+best_ask)/2
spread = best_ask - best_bid

In [12]:
best_ask, best_bid, midpoint, spread

(27151.5, 27151.0, 27151.25, 0.5)

In [24]:
import numpy as np
random = np.random.RandomState(seed=1)
print(random.randint(0,100))

37


In [55]:
from datetime import datetime as dt

now = dt.now()
now.strftime("%Y_%m_%d"+"_at_"+"%H_%M")

'2023_06_03_at_10_04'